In [1]:
import copy
import random

In [ ]:
windyworld = [
    ['_', '_', '_', '_', '_', '_', 'X', 'X'],
    ['_', '_', '_', '_', '_', '_', 'X', 'X'],
    ['_', '_', '_', 'X', '_', '_', 'X', 'X'],
    ['_', '_', '_', 'X', '_', '_', 'X', 'X'],
    ['_', '_', '_', '_', '_', '_', 'X', 'X'],
    ['_', '_', '_', '_', '_', '_', 'X', 'X'],
    ['_', '_', 'X', '_', 'X', '_', '_', '1'],
    ['_', '_', 'X', '_', 'X', '_', '_', '_']
]
wind = ['v', '', 'v', '^', '', 'v', '^', '']

DISCOUNT = .9
LEARNING_RATE = .1
CONVERGENCE_THRESHOLD = .001


# Q-Learning ONLY
def greedy_policy(q, player_X, player_Y):

    max_q = max(q[player_Y][player_X].values())
    max_actions = [action for action, value in q[player_Y][player_X].items() if value == max_q]
    return random.choice(max_actions)


def epsilon_greedy_policy(q, player_X, player_Y, epsilon=0.1):

    if random.random() < epsilon:
        return random.choice(['N', 'E', 'S', 'W'])
    else:
        max_q = max(q[player_Y][player_X].values())
        max_actions = [action for action, value in q[player_Y][player_X].items() if value == max_q]
        return random.choice(max_actions)
    

def take_a(windyworld, wind, a, player_X, player_Y):

    direction_map = {
        'N': (0, -1),  # Move North (decrease Y)
        'S': (0, 1),   # Move South (increase Y)
        'E': (1, 0),   # Move East (increase X)
        'W': (-1, 0),  # Move West (decrease X)
    }

    delta_X, delta_Y = direction_map[a]
    new_X = player_X + delta_X
    new_Y = player_Y + delta_Y

    # Check if the initial movement is within bounds
    if 0 <= new_X < len(windyworld[0]) and 0 <= new_Y < len(windyworld):
        wind_map = {
            '': 0,   # No wind
            '^': -1, # Upward wind
            'v': 1   # Downward wind
        }
        new_Y_wind = new_Y + wind_map.get(wind[new_Y], 0)
        if 0 <= new_Y_wind < len(windyworld):
            return new_X, new_Y_wind

    return player_X, player_Y


def observe_r(windyworld, player_X, player_Y):

    reward_map = {
        '1': 1000,
        '_': -1,
        'X': -10,
    }
    return reward_map[windyworld[player_Y][player_X]]

In [ ]:
'''
Q_learning is not SARSA with greedy, there are key differences:

SARSA evaluates S based on S' (given A) and A' selected from epsilon-greedy and goes along that path.
Updates for Q values are based on current (and possibly non optimal) state path.

Q_learning evaluates S based on S' (given A) and A' selected from greedy, but may not follow that exact path.
Updates for Q values are based on best (and optimal, but possibly not current) state path. 
'''
def Q_learning(windyworld, wind, lr=LEARNING_RATE, d=DISCOUNT, ct=CONVERGENCE_THRESHOLD, min_episode_count=1000):

    q = [[{'N': 0, 'E': 0, 'S': 0, 'W': 0} for _ in r] for r in windyworld]
